In [25]:
%cd ~/grade_prediction/
%pwd

/home/yuhao/grade_prediction


'/home/yuhao/grade_prediction'

In [26]:
import numpy as np
import pandas as pd

# Initialise Model Manager

In [27]:
from ml.dataset import load_student_grade_dataset
from ml.model import PytorchModelManager

dataset = load_student_grade_dataset()
model_manager = PytorchModelManager(dataset)

model_manager.train(epoch=80)

Epoch: 0, loss=0.574, train_accuracy=0.297, test_accuracy=0.190
Epoch: 1, loss=0.591, train_accuracy=0.408, test_accuracy=0.291
Epoch: 2, loss=0.567, train_accuracy=0.437, test_accuracy=0.354
Epoch: 3, loss=0.547, train_accuracy=0.459, test_accuracy=0.354
Epoch: 4, loss=0.535, train_accuracy=0.424, test_accuracy=0.278
Epoch: 5, loss=0.520, train_accuracy=0.481, test_accuracy=0.380
Epoch: 6, loss=0.507, train_accuracy=0.516, test_accuracy=0.367
Epoch: 7, loss=0.583, train_accuracy=0.598, test_accuracy=0.418
Epoch: 8, loss=0.473, train_accuracy=0.623, test_accuracy=0.443
Epoch: 9, loss=0.529, train_accuracy=0.522, test_accuracy=0.418
Epoch: 10, loss=0.458, train_accuracy=0.627, test_accuracy=0.519
Epoch: 11, loss=0.451, train_accuracy=0.646, test_accuracy=0.481
Epoch: 12, loss=0.456, train_accuracy=0.649, test_accuracy=0.456
Epoch: 13, loss=0.435, train_accuracy=0.623, test_accuracy=0.532
Epoch: 14, loss=0.439, train_accuracy=0.677, test_accuracy=0.532
Epoch: 15, loss=0.422, train_accura

In [28]:
x_test = dataset.get_test_X(preprocess=False).iloc[1]
x_test

school              GP
sex                  F
age                 17
address              R
Mjob           at_home
Fjob          services
guardian        father
traveltime           1
studytime            3
paid               yes
nursery             no
internet           yes
absences             0
G1                  11
G2                  11
Name: 318, dtype: object

In [29]:
model_manager.report(x=[x_test])

(  school sex   age address     Mjob      Fjob guardian  traveltime  studytime  \
 0     GP   F  17.0       R  at_home  services   father         1.0        3.0   
 
   paid nursery internet  absences    G1    G2 G3 G3_pred  
 0  yes      no      yes       0.0  11.0  11.0  D       B  ,
 array([[0.00084797, 0.18796104, 0.5667963 , 0.03899765]], dtype=float32))

In [6]:
data_df = dataset.data
features = dataset.features
res = []

for feature in features:
    if feature in dataset.numerical_features:
        feat_min = data_df[feature].min()
        feat_max = data_df[feature].max()

        feat_bins = 4
        labels = list(range(feat_min, feat_max + 1, int((feat_max + 1 - feat_min) / feat_bins)))
        if feat_max not in labels:
            labels.append(feat_max)
        
        feat_obj = {
          'name': feature.upper(),
          'type': 'numerical',
          'min': feat_min,
          'max': feat_max,
          'classes': labels,
          'values': data_df[feature].value_counts(bins = labels).tolist()
        }

    elif feature in dataset.categorical_features:
        category_value_count = data_df[feature].value_counts()
        feat_obj = {
          'name': feature.upper(),
          'type': 'categorical',
          'classes': category_value_count.index.tolist(),
          'values': category_value_count.values.tolist()
        }
    res.append(feat_obj)

In [7]:
res

[{'name': 'SCHOOL',
  'type': 'categorical',
  'classes': ['GP', 'MS'],
  'values': [349, 46]},
 {'name': 'SEX',
  'type': 'categorical',
  'classes': ['F', 'M'],
  'values': [208, 187]},
 {'name': 'AGE',
  'type': 'numerical',
  'min': 15,
  'max': 22,
  'classes': [15, 17, 19, 21, 22],
  'values': [284, 106, 4, 1]},
 {'name': 'ADDRESS',
  'type': 'categorical',
  'classes': ['U', 'R'],
  'values': [307, 88]},
 {'name': 'MJOB',
  'type': 'categorical',
  'classes': ['other', 'services', 'at_home', 'teacher', 'health'],
  'values': [141, 103, 59, 58, 34]},
 {'name': 'FJOB',
  'type': 'categorical',
  'classes': ['other', 'services', 'teacher', 'at_home', 'health'],
  'values': [217, 111, 29, 20, 18]},
 {'name': 'GUARDIAN',
  'type': 'categorical',
  'classes': ['mother', 'father', 'other'],
  'values': [273, 90, 32]},
 {'name': 'TRAVELTIME',
  'type': 'numerical',
  'min': 1,
  'max': 4,
  'classes': [1, 2, 3, 4],
  'values': [364, 23, 8]},
 {'name': 'STUDYTIME',
  'type': 'numerical',

In [51]:
data_df = dataset.data
feature = 'age'
feat_min = data_df[feature].min()
feat_max = data_df[feature].max()

feat_bins = 4
labels = list(range(feat_min, feat_max + 1, int((feat_max + 1 - feat_min) / feat_bins)))
if feat_max not in labels:
    labels.append(feat_max)
data_df[feature].value_counts(bins = labels).tolist()

[284, 106, 4, 1]

In [48]:
list(range(feat_min, feat_max + 1, int((feat_max + 1 - feat_min) / feat_bins)))

[15, 17, 19, 21]

In [33]:
dataset._data['age'].describe()

count    395.000000
mean      16.696203
std        1.276043
min       15.000000
25%       16.000000
50%       17.000000
75%       18.000000
max       22.000000
Name: age, dtype: float64

In [20]:
dataset.target

'G3'

In [11]:
np.array(dataset.dummy_target)

['G3_D', 'G3_C', 'G3_A', 'G3_B']

In [19]:
(np.array(dataset.dummy_target) == "G3_A") * 1

array([0, 0, 1, 0])

# Initialise Engine

In [30]:
from ml.cf_engine.engine import CFEnginePytorch
engine = CFEnginePytorch(dataset, model_manager)

In [31]:
query_instance = dataset._data.iloc[1]
query_instance

school             GP
sex                 F
age                17
address             U
Mjob          at_home
Fjob            other
guardian       father
traveltime          1
studytime           2
paid               no
nursery            no
internet          yes
absences            4
G1                  5
G2                  5
G3                  D
Name: 1, dtype: object

In [32]:
import json
setting = json.loads("{\"changeable_attr\":[\"Mjob\",\"Fjob\",\"guardian\",\"traveltime\",\"studytime\",\"paid\",\"nursery\",\"internet\",\"absences\",\"G1\",\"G2\"],\"cf_range\":{\"school\":{\"name\":\"school\",\"categories\":[\"GP\",\"MS\"]},\"sex\":{\"name\":\"sex\",\"categories\":[\"F\",\"M\"]},\"age\":{\"name\":\"age\",\"extent\":[8,30],\"min\":8,\"max\":30},\"address\":{\"name\":\"address\",\"categories\":[\"U\",\"R\"]},\"Mjob\":{\"name\":\"Mjob\",\"categories\":[\"at_home\",\"health\",\"services\",\"teacher\",\"other\"]},\"Fjob\":{\"name\":\"Fjob\",\"categories\":[\"at_home\",\"health\",\"services\",\"teacher\",\"other\"]},\"guardian\":{\"name\":\"guardian\",\"categories\":[\"mother\",\"father\",\"other\"]},\"traveltime\":{\"name\":\"traveltime\",\"extent\":[0,4],\"min\":0,\"max\":4},\"studytime\":{\"name\":\"studytime\",\"extent\":[0,0.8],\"min\":0,\"max\":0.8},\"paid\":{\"name\":\"paid\",\"categories\":[\"yes\",\"no\"]},\"nursery\":{\"name\":\"nursery\",\"categories\":[\"yes\",\"no\"]},\"internet\":{\"name\":\"internet\",\"categories\":[\"yes\",\"no\"]},\"absences\":{\"name\":\"absences\",\"extent\":[0,0],\"min\":0,\"max\":0},\"G1\":{\"name\":\"G1\",\"extent\":[0,20],\"min\":0,\"max\":20},\"G2\":{\"name\":\"G2\",\"extent\":[0,20],\"min\":0,\"max\":20}},\"num\":12,\"k\":9,\"desired_class\":[[0,0,0,1],[0,0,0,1],[0,0,0,1],[0,0,0,1],[0,0,0,1],[0,0,0,1],[0,0,0,1],[0,0,0,1],[0,0,0,1],[0,0,0,1],[0,0,0,1],[0,0,0,1]]}")
setting

{'changeable_attr': ['Mjob',
  'Fjob',
  'guardian',
  'traveltime',
  'studytime',
  'paid',
  'nursery',
  'internet',
  'absences',
  'G1',
  'G2'],
 'cf_range': {'school': {'name': 'school', 'categories': ['GP', 'MS']},
  'sex': {'name': 'sex', 'categories': ['F', 'M']},
  'age': {'name': 'age', 'extent': [8, 30], 'min': 8, 'max': 30},
  'address': {'name': 'address', 'categories': ['U', 'R']},
  'Mjob': {'name': 'Mjob',
   'categories': ['at_home', 'health', 'services', 'teacher', 'other']},
  'Fjob': {'name': 'Fjob',
   'categories': ['at_home', 'health', 'services', 'teacher', 'other']},
  'guardian': {'name': 'guardian',
   'categories': ['mother', 'father', 'other']},
  'traveltime': {'name': 'traveltime', 'extent': [0, 4], 'min': 0, 'max': 4},
  'studytime': {'name': 'studytime', 'extent': [0, 0.8], 'min': 0, 'max': 0.8},
  'paid': {'name': 'paid', 'categories': ['yes', 'no']},
  'nursery': {'name': 'nursery', 'categories': ['yes', 'no']},
  'internet': {'name': 'internet', '

In [33]:
res = engine.generate_counterfactual_examples([query_instance], setting=setting).valid
res.values.tolist()

TypeError: tuple indices must be integers or slices, not str

In [7]:
res

,school,sex,age,address,famsize,Pstatus,Medu,Fedu,Mjob,Fjob,...,freetime,goout,Dalc,Walc,health,absences,G1,G2,G3,G3_pred
0,GP,F,17.0,U,GT3,T,1.0,1.0,at_home,other,...,3.0,3.0,1.0,1.0,3.0,23.0,9.0,13.0,B,B
1,GP,F,17.0,U,GT3,T,1.0,1.0,at_home,other,...,3.0,3.0,1.0,1.0,3.0,20.0,14.0,6.0,B,B
3,GP,F,17.0,U,GT3,T,1.0,1.0,at_home,other,...,3.0,3.0,1.0,1.0,3.0,4.0,11.0,10.0,B,B
4,GP,F,17.0,U,GT3,T,1.0,1.0,at_home,other,...,3.0,3.0,1.0,1.0,3.0,9.0,11.0,11.0,B,B
5,GP,F,17.0,U,GT3,T,1.0,1.0,health,other,...,3.0,3.0,1.0,1.0,3.0,4.0,8.0,11.0,B,B
6,GP,F,18.0,U,GT3,T,1.0,1.0,health,other,...,3.0,3.0,1.0,1.0,3.0,4.0,6.0,14.0,B,B
7,GP,F,17.0,U,GT3,T,1.0,1.0,health,other,...,3.0,3.0,1.0,1.0,3.0,4.0,5.0,14.0,B,B
8,GP,F,17.0,U,GT3,T,1.0,1.0,health,other,...,3.0,3.0,1.0,1.0,3.0,4.0,12.0,6.0,B,B
9,GP,F,17.0,U,GT3,T,1.0,1.0,health,other,...,3.0,3.0,1.0,1.0,3.0,4.0,10.0,8.0,B,B
10,GP,F,17.0,U,GT3,T,1.0,1.0,health,other,...,3.0,3.0,1.0,1.0,3.0,0.0,8.0,9.0,B,B


In [11]:
pd.DataFrame([query_instance, res.iloc[0]])

,school,sex,age,address,famsize,Pstatus,Medu,Fedu,Mjob,Fjob,...,freetime,goout,Dalc,Walc,health,absences,G1,G2,G3,G3_pred
1,GP,F,17.0,U,GT3,T,1.0,1.0,at_home,other,...,3.0,3.0,1.0,1.0,3.0,4.0,5.0,5.0,D,NaN
0,GP,F,17.0,U,GT3,T,1.0,1.0,at_home,other,...,3.0,3.0,1.0,1.0,3.0,23.0,9.0,13.0,B,B


In [26]:
x, x_dash = query_instance.to_list(), res.iloc[2].to_list()[:-1]

for i in range(len(x)):
  if x[i] != x_dash[i]:
    print(dataset.columns[i]+":", x[i], "->", x_dash[i])

G1: 5 -> 11.0
G2: 5 -> 10.0
G3: D -> B


In [17]:
dataset.columns

['school',
 'sex',
 'age',
 'address',
 'famsize',
 'Pstatus',
 'Medu',
 'Fedu',
 'Mjob',
 'Fjob',
 'reason',
 'guardian',
 'traveltime',
 'studytime',
 'failures',
 'schoolsup',
 'famsup',
 'paid',
 'activities',
 'nursery',
 'higher',
 'internet',
 'romantic',
 'famrel',
 'freetime',
 'goout',
 'Dalc',
 'Walc',
 'health',
 'absences',
 'G1',
 'G2',
 'G3']